In [2]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler
import os

In [2]:
df_ov_clinical = pd.read_excel("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_Zhang_TCGA_CPTAC_OV/mmc2.xlsx")
df_ov_clinical.head(10)
len(df_ov_clinical)

/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


174

In [3]:
selected_stages= ["IIIA","IIIB","IIIC","IV"]
df_ov_clinical = df_ov_clinical[df_ov_clinical["tumor_stage"].isin(selected_stages)]

In [4]:
df_ov_clinical["PlatinumStatus"].value_counts()
df_ov_clinical = df_ov_clinical[df_ov_clinical['PlatinumStatus'] != "Not available"]

# remove not avalable... 
df_ov_clinical['label'] = df_ov_clinical['PlatinumStatus'].map({'Sensitive': 1, 'Resistant': 0})
df_ov_clinical['label'].value_counts()

1    85
0    35
Name: label, dtype: int64

#### Load proteomic data

In [5]:
tcga_proteomic = pd.read_excel("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_Zhang_TCGA_CPTAC_OV/mmc3-2.xlsx")

In [6]:
tcga_prots = tcga_proteomic["hgnc_symbol"].to_list()

In [7]:
# comapre to training dataset here in analysis
# load other to find amount of overlapping prot and phospoprots.
gloabal_prot = pd.read_csv("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_processed_data/FFPE_discovery_globalprotein_imputed.tsv",delimiter='\t')
HGSOC_prots = gloabal_prot["HGNC symbol"].to_list()

In [8]:
HGSOC_prots_set = ['ABCB8', 'ACADSB', 'ACOT7', 'AIFM1', 'ARNT', 'ATM',
       'ATP6V1D', 'BCL2L1', 'BIRC2', 'CA2', 'CAMK2D', 'CARMIL1', 'CASP7',
       'CCDC167', 'CDKN1A', 'CDKN1B', 'CHUK', 'COX6A1', 'COX6C', 'COX7A2',
       'COX7C', 'CPOX', 'EGFR', 'FDX1', 'GLUD1', 'GOLPH3L', 'GPX1', 'GPX4',
       'HACL1', 'HADH', 'IDH1', 'IDI1', 'IL4I1', 'KEAP1', 'LDHA', 'LGALS1',
       'LTA4H', 'MTDH', 'MYD88', 'NBN', 'NCAPH2', 'NDUFB3', 'NFKB1', 'NFKB2',
       'PDK1', 'PECR', 'PLIN2', 'PTGES2', 'RAB25', 'RELA', 'RHOA', 'RIPK1',
       'RPS6KB2', 'S100A10', 'SENP1', 'STAT3', 'TALDO1', 'TFRC', 'TGM2',
       'TOP3B', 'TPMT', 'VCAM1', 'XIAP', 'YWHAG']


set1 = set(tcga_prots)
set2 = set(HGSOC_prots_set)

# Find intersection
intersection = set1 & set2

# Convert back to list
intersecting_list = list(intersection)

len(intersecting_list)
# 7621 intesecting.... 60/64 in dicsovery set... 

60

# Norm , Impute and Merge proteomics onto clinical

In [9]:
from sklearn.impute import SimpleImputer, KNNImputer


# transopose and organise 
tcga_proteomic_t = tcga_proteomic.T
tcga_proteomic_t.columns = tcga_proteomic_t.iloc[1]
tcga_proteomic_t = tcga_proteomic_t.iloc[2:]
# impute 
# tcga_proteomic_t.index
knn_imputer = KNNImputer(n_neighbors=5)

scaler = StandardScaler()
# Fit and transform the DataFrame
tcga_proteomic_data = pd.DataFrame(scaler.fit_transform(tcga_proteomic_t), columns=tcga_proteomic_t.columns, index=tcga_proteomic_t.index)

tcga_proteomic_data = pd.DataFrame(knn_imputer.fit_transform(tcga_proteomic_data), columns=tcga_proteomic_data.columns, index =tcga_proteomic_data.index)

tcga_proteomic_data["bcr_patient_barcode"] = tcga_proteomic_t.index.str.split('-', 1).str[1]
tcga_proteomic_data.head()

/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['datetime', 'float', 'str']. An error will be raised in 1.2.
  FutureWarning,
/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['datetime', 'float', 'str']. An error will be raised in 1.2.
  FutureWarning,
/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['datetime', 'float', 'str']. An error will be raised in 1.2.
  FutureWarning,
/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got fe

hgnc_symbol,A2M,ACADM,ACADS,ACADVL,ACAT1,PSEN1,ADA,ADSL,AGA,AGT,...,MT-ND2,MT-CO1,MT-CO2,MT-ATP8,MT-ATP6,MT-CO3,MT-ND4,MT-ND5,MT-CYB,bcr_patient_barcode
PNNL-TCGA-09-1664,0.329787,-0.089153,0.170035,0.231383,0.320671,-1.783330,-1.787210,-1.039627,-0.794098,0.625866,...,-0.066533,0.402725,2.093747,-0.170579,0.036133,0.473987,-0.143183,0.204756,0.855386,TCGA-09-1664
PNNL-TCGA-13-1484,-0.629509,0.944590,-0.127556,-0.597598,0.531937,-1.009674,2.102763,0.901624,1.605626,-1.028414,...,-1.358740,-0.702376,0.350870,1.703491,0.870127,0.683079,-0.248484,3.860032,0.028845,TCGA-13-1484
PNNL-TCGA-13-1488,1.207150,0.267773,0.979108,0.403672,0.733048,-2.131350,0.963103,0.713942,-1.502288,0.326415,...,-0.460440,0.269568,0.809037,-0.093694,0.326904,0.658271,0.807531,1.496455,0.428676,TCGA-13-1488
PNNL-TCGA-13-1489,-0.239638,0.675980,0.222113,2.491108,2.108964,0.262226,-0.067715,1.296598,-0.731602,0.122652,...,-0.400554,-0.759737,-1.458776,1.465628,-0.283940,-0.344100,-1.046849,-1.924024,-0.539971,TCGA-13-1489
PNNL-TCGA-13-1494,0.925407,-0.628408,-0.432586,-0.779874,-2.944690,-0.018193,-0.469402,0.431018,0.980886,-0.101015,...,0.415860,0.119793,1.135971,-0.360389,1.483225,0.212162,1.868202,1.029291,-0.026735,TCGA-13-1494


### Load Phospo-proteomic_data

In [4]:
tcga_phospoproteomic = pd.read_excel("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_Zhang_TCGA_CPTAC_OV/mmc3-2.xlsx",sheet_name="FullOvarianPhosphoproteomeData")
tcga_phospoproteomic.head()



# # transopose and organise 
# tcga_phospoproteomic_t = tcga_phospoproteomic.T
# tcga_phospoproteomic_t.columns = tcga_phospoproteomic_t.iloc[1]
# tcga_phospoproteomic_t = tcga_phospoproteomic_t.iloc[2:]
# # impute 
# # tcga_phospoproteomic_t.index
# knn_imputer = KNNImputer(n_neighbors=5)

# scaler = StandardScaler()
# # Fit and transform the DataFrame
# tcga_phospoproteomic_data = pd.DataFrame(scaler.fit_transform(tcga_phospoproteomic_t), columns=tcga_phospoproteomic_t.columns, index=tcga_phospoproteomic_t.index)

# tcga_phospoproteomic_data = pd.DataFrame(knn_imputer.fit_transform(tcga_phospoproteomic_data), columns=tcga_phospoproteomic_data.columns, index=tcga_phospoproteomic_data.index)

# tcga_phospoproteomic_data["bcr_patient_barcode"] = tcga_phospoproteomic_t.index.str.split('-', 1).str[1]
# tcga_phospoproteomic_data.head()


In [6]:
tcga_phospoproteomic.head()
len(tcga_phospoproteomic) # not in same format as train set... --> maybe best not to use at moment.

42825

In [10]:
merged = pd.merge(tcga_proteomic_data,df_ov_clinical, on='bcr_patient_barcode')
merged.head()

,A2M,ACADM,ACADS,ACADVL,ACAT1,PSEN1,ADA,ADSL,AGA,AGT,...,vital_status,daystodeath_or_LFU,Prog_or_Recur,daystotumorprog-recur-LFU,PlatinumStatus,PlatinumFreeInterval (mos),ethnicity,race,year_of_diagnosis,label
0,-0.629509,0.944590,-0.127556,-0.597598,0.531937,-1.009674,2.102763,0.901624,1.605626,-1.028414,...,DECEASED,3728,Yes,496,Sensitive,10.1,NOT HISPANIC OR LATINO,WHITE,2000,1
1,1.207150,0.267773,0.979108,0.403672,0.733048,-2.131350,0.963103,0.713942,-1.502288,0.326415,...,DECEASED,2154,Yes,351,Sensitive,6.0,NOT HISPANIC OR LATINO,WHITE,2002,1
2,0.282348,0.253173,1.115470,1.145510,0.537811,0.422215,1.826634,0.845054,1.449441,-0.665003,...,DECEASED,2154,Yes,351,Sensitive,6.0,NOT HISPANIC OR LATINO,WHITE,2002,1
3,-0.239638,0.675980,0.222113,2.491108,2.108964,0.262226,-0.067715,1.296598,-0.731602,0.122652,...,DECEASED,2553,Yes,806,Sensitive,17.5,NOT HISPANIC OR LATINO,WHITE,2002,1
4,-0.431909,-0.192069,0.382654,2.501345,1.642925,0.431229,-0.416080,0.906681,-0.368395,-0.461536,...,DECEASED,2553,Yes,806,Sensitive,17.5,NOT HISPANIC OR LATINO,WHITE,2002,1


#### setup HGSOC proteomics data and train classifier with overlapping prots. Then test of tcga data... with label
### first prep dataframe with label slides and signatures.


### Set up dummy learning task to see if there is some signal or corrolation?....

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, roc_auc_score


train_df, test_df  = train_test_split(merged, test_size=0.2, random_state=42)

X_train = train_df[intersecting_list]
X_test =  test_df[intersecting_list]
y_train = train_df['label']
y_test = test_df['label']

# Initialize and train the Random Forest classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

# Make predictions on the test set
rf_pred = rf_clf.predict(X_test)
rf_pred_proba = rf_clf.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Evaluate the Random Forest model.
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_roc_auc = roc_auc_score(y_test, rf_pred_proba)
print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest ROC AUC: {rf_roc_auc}")

Random Forest Accuracy: 0.7241379310344828
Random Forest ROC AUC: 0.5892857142857143


In [16]:
print(intersecting_list)

['RAB25', 'BCL2L1', 'HADH', 'NFKB2', 'COX7A2', 'COX7C', 'TPMT', 'GOLPH3L', 'LTA4H', 'COX6C', 'IDH1', 'YWHAG', 'S100A10', 'COX6A1', 'NDUFB3', 'TGM2', 'CDKN1B', 'NFKB1', 'CAMK2D', 'IL4I1', 'FDX1', 'VCAM1', 'ATM', 'NCAPH2', 'ABCB8', 'IDI1', 'PLIN2', 'ATP6V1D', 'GPX4', 'CA2', 'RELA', 'GLUD1', 'TOP3B', 'RPS6KB2', 'KEAP1', 'LGALS1', 'MTDH', 'AIFM1', 'RHOA', 'CASP7', 'PTGES2', 'TFRC', 'CHUK', 'GPX1', 'PDK1', 'STAT3', 'PECR', 'TALDO1', 'XIAP', 'ACADSB', 'CPOX', 'ARNT', 'BIRC2', 'ACOT7', 'HACL1', 'MYD88', 'EGFR', 'RIPK1', 'NBN', 'LDHA']


### Merge the wsi slide paths for testing. 

In [12]:
folder_path = '/tank/WSI_data/Ovarian_WSIs/TCGA-OV/slides'

# List all filenames in the folder
filenames = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Extract TCGA identifiers from filenames and create a new DataFrame
# Assuming the TCGA ID is always at the start of the filename up to the third '-'
file_df = pd.DataFrame({'Filename': filenames})
file_df['bcr_patient_barcode'] = file_df['Filename'].apply(lambda x: '-'.join(x.split('-')[:3]))

file_df['bcr_patient_barcode']

proteomics_and_wsi = merged.merge(file_df, on='bcr_patient_barcode', how='left')

In [13]:
proteomics_and_wsi["Filename"] = proteomics_and_wsi["Filename"].str[:-4]

In [14]:
len(proteomics_and_wsi)

# now change column namues to be appropriate 

proteomics_and_wsi = proteomics_and_wsi.rename(columns={"bcr_patient_barcode":"case_id","Filename":"slide_id"})

In [15]:
proteomics_and_wsi.to_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/TCGA_platinum_reposnce_test_set.csv",index=None)